In [1]:
from fastai.vision.all import *
import fastai
print(fastai.__version__)

2.7.19


In [2]:
path = Path('/kaggle/working/database')
path

Path('/kaggle/working/database')

In [3]:
if not path.exists():
    print("download again....")
    path_download = untar_data(URLs.IMAGENETTE)
    shutil.move(str(path_download),str(path))

download again....


In [4]:
dblock = DataBlock(blocks=(ImageBlock(),CategoryBlock()),
                  get_items=get_image_files,
                  get_y=parent_label,
                  item_tfms=Resize(460),
                  batch_tfms=aug_transforms(size=224,min_scale=0.75))

In [5]:
dls=dblock.dataloaders(path,bs=64)

In [6]:
model=xresnet50()
learn=Learner(dls,model,loss_func=CrossEntropyLossFlat(),metrics=accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.632681,2.708280,0.390590,02:26
1,1.250909,1.158778,0.627707,02:43
2,0.985891,1.104755,0.653846,02:45
3,0.739711,0.673147,0.794996,02:46
4,0.588607,0.568099,0.824122,02:44


In [7]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([0.4393, 0.4367, 0.4034], device='cuda:0'),
 TensorImage([0.2778, 0.2722, 0.2886], device='cuda:0'))

In [12]:
def get_dls(bs,size):
    dblock = DataBlock(blocks=(ImageBlock,CategoryBlock),
                      get_items=get_image_files,
                      get_y=parent_label,
                      item_tfms=Resize(460),
                      batch_tfms=[*aug_transforms(size=size,min_scale=0.75),Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(path,bs=bs)

In [13]:
dls = get_dls(64,224)

In [14]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([-0.1147, -0.0448,  0.0154], device='cuda:0'),
 TensorImage([1.2026, 1.2307, 1.3252], device='cuda:0'))

In [15]:
model=xresnet50()
learn=Learner(dls,model,loss_func=CrossEntropyLossFlat(),metrics=accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.638336,1.717343,0.515310,02:48
1,1.246444,1.218339,0.599701,02:44
2,0.968762,1.185548,0.664675,02:43
3,0.763226,0.666752,0.792009,02:43
4,0.595982,0.567482,0.817774,02:45


In [17]:
dls = get_dls(128,128)
learn = Learner(dls,xresnet50(),loss_func=CrossEntropyLossFlat(),metrics=accuracy)
learn.fit_one_cycle(4,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.839384,1.779263,0.465646,01:19
1,1.315024,1.316212,0.577296,01:18
2,0.978945,0.788455,0.737864,01:19
3,0.767880,0.681953,0.784541,01:19


In [18]:
learn.dls = get_dls(64,224)
learn.fine_tune(5,1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.825445,1.086587,0.652353,02:47


epoch,train_loss,valid_loss,accuracy,time
0,0.669369,0.789033,0.743465,02:45
1,0.670225,1.098828,0.683346,02:46
2,0.618711,0.645475,0.798357,02:45
3,0.488514,0.511766,0.840926,02:43
4,0.447491,0.476561,0.846527,02:44


In [19]:
preds,targs = learn.tta()
accuracy(preds,targs).item()

0.8547423481941223

In [20]:
image2,target2 = dataset[randint90,len(dataset)]
t = random_float(0.5,1.0)
new_image = t* image1 + (1 - t) * image2
new_target = t * target1 + (1 - t) * target2

NameError: name 'dataset' is not defined

In [6]:

model=xresnet50()
learn=Learner(dls,model,loss_func=CrossEntropyLossFlat(),metrics=accuracy,cbs=MixUp)
# learn=Learner(dls,model,loss_func=CrossEntropyLossFlat(),metrics=accuracy,cbs=SimpleMixup())
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.973720,1.699164,0.445855,02:29
1,1.623735,1.397621,0.557506,02:35
2,1.436840,1.245876,0.594473,02:36
3,1.264101,0.874000,0.724421,02:36
4,1.147557,0.698235,0.796863,02:36


In [33]:
model=xresnet50()
learn=Learner(dls,model,loss_func=LabelSmoothingCrossEntropy(),metrics=accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.738256,2.772862,0.465273,01:19
1,2.256537,2.378651,0.533607,01:19
2,1.972466,2.021921,0.657207,01:19
3,1.776993,1.685234,0.777819,01:19
4,1.649993,1.591090,0.810680,01:19


In [1]:
import fastai
import fastai.vision.all as fv
import fastai.callback as fc

print("fastai version:", fastai.__version__)

print("\nAttributes in fastai.vision.all:")
print([x for x in dir(fv) if "Mix" in x])

print("\nAttributes in fastai.callback:")
for sub in dir(fc):
    if not sub.startswith("_"):
        try:
            mod = __import__(f"fastai.callback.{sub}", fromlist=["*"])
            mix_things = [x for x in dir(mod) if "Mix" in x]
            if mix_things:
                print(f"  {sub}:", mix_things)
        except:
            pass


fastai version: 2.7.19

Attributes in fastai.vision.all:
['CutMix', 'MixHandler', 'MixUp', 'MixedPrecision', 'NonNativeMixedPrecision']

Attributes in fastai.callback:
  all: ['CutMix', 'MixHandler', 'MixUp', 'MixedPrecision', 'NonNativeMixedPrecision']
  channelslast: ['MixedPrecision']
  fp16: ['MixedPrecision', 'NonNativeMixedPrecision']
  mixup: ['CutMix', 'MixHandler', 'MixUp']
  tracker: ['MixedPrecision']
  training: ['MixedPrecision', 'NonNativeMixedPrecision']
